In [1]:
import scipy as sp
from pyscf import gto, dft, lib, mp, cc, scf, tools, ci, fci

In [2]:
import numpy as np
import sys
sys.path.append( './src/' )

In [3]:
matrix_dot = lambda A, B: np.einsum('ij,ij', A, B)

In [4]:
# H    0.7493682,0.0000000,0.4424329
# O    0.0000000,0.0000000,-0.1653507
# H   -0.7493682,0.0000000,0.4424329

# Define Molecule 

- Note first N atoms will be the ACTIVE portion of the molecule
- the remaining atoms will be the environment

In [141]:
set([1,2,3]) 

{1, 2, 3}

In [139]:
N_active_atoms = 2

geometry = [
['H', (0.7493682,0.0000000,0.4424329)],
['O', (0.0000000,0.0000000,-0.1653507)],
    
['H', (-0.7493682,0.0000000,0.4424329)]
]

# geometry = [
# ['Be@1', (0.0000000,0.0000000,0.0000000)],
# ['H@1', (0.0000000,0.0000000,+1)],
    
# ['H#', (0.0000000,0.0000000,-1)]
# ]


# geometry = """3
 
# H	0.7493682	0.0	0.4424329
# O	0.0	0.0	-0.1653507
# H	-0.7493682	0.0	0.4424329
# """

In [131]:
if isinstance(geometry, str):
    # if already xzy format
    xyz_geom = geometry
else:
    # convert to xyz format
    n_atoms = len(geometry)
    xyz_geom=f'{n_atoms}'
    xyz_geom+='\n \n'
    for atom, xyz in geometry:
        xyz_geom+= f'{atom}\t{xyz[0]}\t{xyz[1]}\t{xyz[2]}\n'

print(xyz_geom)

3
 
H	0.7493682	0.0	0.4424329
O	0.0	0.0	-0.1653507
H	-0.7493682	0.0	0.4424329



In [126]:
import py3Dmol
import rdkit
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
rdkit.Chem.Draw.IPythonConsole.ipython_3d = True  # enable py3Dmol inline visualization

width= 400
height = 400

view = py3Dmol.view(width=width, height=height)
view.addModel(xyz_geom, "xyz")
view.setStyle({'stick':{}})
view.zoomTo()
view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

# Define Params

- what level to treat active and envirnoment systems

In [132]:
## define params

basis =  'STO-6G' # '6-31g' #
low_level_xc_functional_or_HF = 'lda, vwn' #'hf'
high_level_xc_functional = 'b3lyp'

low_level_method = 'rhf'
high_level_ref = 'rhf'
high_level_method = 'mp2'

# Build the GLOBAL (full) molecule

In [138]:
full_system_mol = gto.Mole(atom= geometry,
                      basis=basis,
                       charge=0,
                       #spin=0,
                      )
full_system_mol.build()
full_system_mol.atom

ValueError: invalid literal for int() with base 10: '0.7493682'

In [10]:
full_system_mol.verbose = 1
full_system_mol.max_memory = 8_000 #MB

# 1. Run Supersystem Calculation (cheap method)

In [11]:
full_system_scf = scf.RKS(full_system_mol)
full_system_scf.verbose=1
full_system_scf.max_memory= 8_000
full_system_scf.conv_tol = 1e-6
full_system_scf.xc = low_level_xc_functional_or_HF
full_system_scf.kernel()

-75.45216076573016

In [12]:
full_system_scf.e_tot

-75.45216076573016

In [13]:
full_system_scf.conv_check

True

In [14]:
two_e_term_total = full_system_scf.get_veff()
e_xc_total = two_e_term_total.exc
v_xc_total = two_e_term_total - full_system_scf.get_j() 

In [15]:
# full_system_scf.get_fock()
# full_system_scf.get_hcore()

In [16]:
# full_system_scf.mo_coeff.shape

# Run expensive CI calc to compare all results too

In [17]:
HF_scf = scf.RHF(full_system_mol)
HF_scf.verbose=1
HF_scf.max_memory= 8_000
HF_scf.conv_tol = 1e-6
HF_scf.kernel()

my_fci = fci.FCI(HF_scf).run()
print('E(UHF-FCI) = %.12f' % my_fci.e_tot)

# myci = ci.CISD(HF_scf).run() # this is UCISD
# print('UCISD total energy = ', myci.e_tot)

E(UHF-FCI) = -75.731522191956


# 2. Localise orbitals

### Background

The overlap matrix is:

$$S_{\mu \nu} =  \int d\vec{r}_{1} \phi_{\mu}(1)^{*}\phi_{\nu}(1)$$

- $\phi_{\mu}$ are basis functions (defined in basis set)


The unknown molecular orbitals $\psi_{i}$ are expanded as a linear expansion of the $K$ known basis functions:

$$ \psi_{i} =  \sum_{\mu=1}^{K} C_{\mu i} \psi_{\mu}$$


$C$ is a $K \times K$ matrix of expansion coefficients $C_{\mu i}$. The columns of $C$ describe the molecular orbitals!

We can find the total number of electrons $N$ in the system by:

$$ N =  2 \sum_{a}^{N/2}\int d\vec{r}  \bigg( \psi_{a}(\vec{r})^{*} \psi_{i}(\vec{r}) \bigg) =  2 \sum_{a}^{N/2} 1$$

- integral gives probablity of finding electron $a$ over all space (must be 1)
- summing over all electrons will give the total number of electrons

The charge density has the following definition:

$$\rho(\vec{r}) = 2 \sum_{a}^{N/2} \bigg( \psi_{a}(\vec{r})^{*} \psi_{i}(\vec{r}) \bigg)$$

- re-write using definition of $\psi_{i}=  \sum_{\mu=1}^{K} C_{\mu i} \psi_{\mu}$

$$\rho(\vec{r}) = 2 \sum_{a}^{N/2} \Bigg( \bigg[ \sum_{\nu}^{K} C_{\nu a}^{*} \phi_{\nu}(\vec{r})^{*} \bigg] \bigg[ \sum_{\mu}^{K} C_{\mu a}\phi_{\mu}(\vec{r}) \bigg] \Bigg)$$

- move things around

$$\rho(\vec{r}) = \sum_{\nu}^{K} \sum_{\mu}^{K} \Big( 2 \sum_{a}^{N/2} C_{\mu a} C_{\nu a}^{*} \Big) \phi_{\mu}(\vec{r}) \phi_{\nu}(\vec{r})^{*} $$

- which is 

$$\rho(\vec{r}) = \sum_{\mu, \nu}^{K} P_{\mu \nu} \phi_{\mu}(\vec{r}) \phi_{\nu}(\vec{r})^{*} $$


- $P_{\mu \nu}$ is known as the density matrix and is:

$$P_{\mu \nu} = 2 \sum_{a}^{N/2} C_{\mu a} C_{\nu a}^{*}$$

Therefore we can also find the total number of electrons in the system by:

$$ N =  2 \sum_{a}^{N/2}\int d\vec{r}  \bigg( \psi_{a}(\vec{r})^{*} \psi_{i}(\vec{r}) \bigg) =  \sum_{\nu}^{K} \sum_{\mu}^{K} \Big( 2 \sum_{a}^{N/2} C_{\mu a} C_{\nu a}^{*} \Big) \int d\vec{r} \phi_{\mu}(\vec{r})  \phi_{\nu}(\vec{r})^{*}$$

- This is simply:

$$N =  \sum_{\nu}^{K} \sum_{\mu}^{K} P_{\mu \nu} S_{\nu \mu}$$

- in an orthognal basis $S$ should be the identity matrix (as it is overlap of matrix of orthogonal orbs), SO WE GET:

$$N =  \sum_{\nu}^{K} \sum_{\mu}^{K} P_{\mu \nu} S_{\nu \mu} = \sum_{\mu}^{K} PS_{\mu \mu} = \mathcal{Tr}(PS)$$

- One can interpret $ PS_{\mu \mu}$ in the above equation as the number of electrons associated with $ \phi_{\mu}$
- This is a **Mulliken population analysis**

# Orbital Localization!

In [18]:
from pyscf import lo

A molecular orbital is usually delocalized, i.e. it has non-negligible amplitude over the whole system rather than only around some atom(s) or bond(s). However, one can choose a unitary rotation 

- When we perform a SCF calculation, one gets an optimized C matrix
    - $C$ is a $K \times K$ matrix of expansion coefficients $C_{\mu i}$
    - The columns of $C$ describe the molecular orbitals!
    - MO i: $ \psi_{i} =  \sum_{\mu=1}^{K} C_{\mu i} \psi_{\mu}$
    
    
- These molecular orbitals are usually **delocalized**
    - non-negligible amplitude over the whole system, rather than only around some atom(s) or bond(s)

- But we know in QM that a given basis choice is NOT unique


- We can therefore perform a unitary rotation on molecular orbitals

$$ \psi_{i} U_{rot} =  \Big( \sum_{\mu=1}^{K} C_{\mu i} \psi_{\mu} \Big) U_{rot} = \psi_{i}^{new}$$
    
    
The idea is to use a rotation such that the resulting orbitals $\psi_{i}^{new}$ are as spatially localized as possible. 


The Pipek-Mezey (PM) [localization](https://notendur.hi.is/hj/papers/paperPipekmezey8.pdf) **maximizes the population charges on the atoms**:

$$ f (U_{rot}) = \sum_{A}^{N_{atoms}} \Bigg( Z_{A} -  \sum_{\mu \text{ on atom } A} PS_{\mu \mu} \Bigg)$$

In [19]:
localization_method = 'ibo'#'spade' 'pipekmezey' 'boys 'ibo'
THRESHOLD = 0.4

S_mat = full_system_scf.get_ovlp()
AO_slice_matrix = full_system_mol.aoslice_by_atom()



if localization_method.lower() == 'spade':
    
    occ_orbs = full_system_scf.mo_coeff[:,full_system_scf.mo_occ>0]
    N_active_AO = AO_slice_matrix[N_active_atoms-1][3]
    
    ao_active_inds = np.arange(N_active_AO)
    
    S_half = sp.linalg.fractional_matrix_power(S_mat, 0.5)
    orthogonal_orbitals = (S_half@occ_orbs)[:N_active_AO, :] # Get rows (the active AO) of orthogonal orbs 
    
    u, singular_values, rotation_matrix = np.linalg.svd(orthogonal_orbitals, full_matrices=True)
    delta_s = singular_values[:-1] - singular_values[1:] # σ_i - σ_(i+1)
    
    n_act_mos = np.argmax(delta_s)+1 # add one due to python indexing
    n_env_mos = len(singular_values) - n_act_mos
    
    act_orbitals = occ_orbs @ rotation_matrix.T[:, :n_act_mos]
    env_orbitals = occ_orbs @ rotation_matrix.T[:, n_act_mos:]
    
    C_matrix_all_localized_orbitals = occ_orbs @ rotation_matrix.T
    
    active_MO_inds  = np.arange(n_act_mos)
    enviro_MO_inds = np.arange(n_act_mos, n_act_mos+n_env_mos)
    
else:
    if localization_method.lower() == 'pipekmezey':
        ### PipekMezey
        PM = lo.PipekMezey(full_system_mol, full_system_scf.mo_coeff)
        PM.pop_method = 'mulliken' # 'meta-lowdin', 'iao', 'becke'
        C_loc = PM.kernel() # includes virtual orbs too!
        C_loc_occ =C_loc[:,full_system_scf.mo_occ>0]
        
    elif localization_method.lower() == 'boys':
        ### Boys
        boys_SCF = lo.boys.Boys(full_system_mol, full_system_scf.mo_coeff)
        C_loc  = boys_SCF.kernel()
        C_loc_occ =C_loc[:,full_system_scf.mo_occ>0]
        
    elif localization_method.lower() == 'ibo':
        ### intrinsic bonding orbs
        #
        mo_occ = full_system_scf.mo_coeff[:,full_system_scf.mo_occ>0]
        iaos = lo.iao.iao(full_system_scf.mol, mo_occ)
        C_loc_occ = lo.ibo.ibo(full_system_scf.mol, mo_occ, locmethod='IBO', iaos=iaos)#.kernel()
    else:
        raise ValueError(f'unknown localization method {localization_method}')
        
    
    # find indices of AO of active atoms
    ao_active_inds = np.arange(AO_slice_matrix[0,2], AO_slice_matrix[N_active_atoms-1,3])
    
    
    MO_AO_overlap = S_mat@C_loc_occ  #  < ϕ_AO_i | ψ_MO_j >
    MO_active_AO_overlap = np.einsum('ij->j', MO_AO_overlap[ao_active_inds]) # sum over rows of active AOs of MOs!
    
    print('\noverlap:', MO_active_AO_overlap)
    print(f'threshold for active part: {THRESHOLD} \n')
    
    active_MO_inds = np.where(MO_active_AO_overlap>THRESHOLD)[0]
    enviro_MO_inds = np.array([i for i in range(C_loc_occ.shape[1]) if i not in active_MO_inds]) # get all non active MOs
    
    
    # define active MO orbs and environment
    act_orbitals = C_loc_occ[:, active_MO_inds] # take MO (columns of C_matrix) that have high dependence from active AOs
    env_orbitals = C_loc_occ[:, enviro_MO_inds]

    C_matrix_all_localized_orbitals = C_loc_occ

    n_act_mos = len(active_MO_inds)
    n_env_mos = len(enviro_MO_inds)
    

    
### Get active and enviro density matrices

dm_active =  2 * act_orbitals @ act_orbitals.T
dm_enviro =  2 * env_orbitals @ env_orbitals.T

print(f'number of active MOs: {n_act_mos}')
print(f'number of enviro MOs: {n_env_mos} \n')


bool_flag_electron_number = np.isclose(np.trace(dm_active@S_mat) + np.trace(dm_enviro@S_mat), full_system_mol.nelectron)
print(f'N_active + N_environment = N is: {bool_flag_electron_number}')

dm_localised_full_system = 2* C_matrix_all_localized_orbitals@ C_matrix_all_localized_orbitals.conj().T
density_flag = np.allclose(dm_localised_full_system, dm_active + dm_enviro)
print(f'y_active + y_enviro = y_total is: {density_flag}')


 Iterative localization: IB/P4/2x2, 5 iter; Final gradient 7.53e-09

overlap: [ 0.11222552  2.45080954 -0.47162607 -1.32205121  0.77644361]
threshold for active part: 0.4 

number of active MOs: 2
number of enviro MOs: 3 

N_active + N_environment = N is: True
y_active + y_enviro = y_total is: True


In [20]:
# from copy import deepcopy

# localization_method = 'ibo'#'spade' 'pipekmezey' 'boys 'ibo'
# THRESHOLD = 0.4

# S_mat = full_system_scf.get_ovlp()
# AO_slice_matrix = full_system_mol.aoslice_by_atom()



# if localization_method.lower() == 'spade':
    
#     occ_orbs = full_system_scf.mo_coeff[:,full_system_scf.mo_occ>0]
#     N_active_AO = AO_slice_matrix[N_active_atoms-1][3]
    
#     ao_active_inds = np.arange(N_active_AO)
    
#     S_half = sp.linalg.fractional_matrix_power(S_mat, 0.5)
#     orthogonal_orbitals = (S_half@occ_orbs)[:N_active_AO, :] # Get rows (the active AO) of orthogonal orbs 
    
#     u, singular_values, rotation_matrix = np.linalg.svd(orthogonal_orbitals, full_matrices=True)
#     delta_s = singular_values[:-1] - singular_values[1:] # σ_i - σ_(i+1)
    
#     n_act_mos = np.argmax(delta_s)+1 # add one due to python indexing
#     n_env_mos = len(singular_values) - n_act_mos
    
#     act_orbitals = occ_orbs @ rotation_matrix.T[:, :n_act_mos]
#     env_orbitals = occ_orbs @ rotation_matrix.T[:, n_act_mos:]
    
#     C_matrix_all_localized_orbitals = occ_orbs @ rotation_matrix.T
    
#     active_MO_inds  = np.arange(n_act_mos)
#     enviro_MO_inds = np.arange(n_act_mos, n_act_mos+n_env_mos)
    
# else:
#     if localization_method.lower() == 'pipekmezey':
#         ### PipekMezey
#         PM = lo.PipekMezey(full_system_mol, full_system_scf.mo_coeff)
#         PM.pop_method = 'mulliken' # 'meta-lowdin', 'iao', 'becke'
#         C_loc = PM.kernel() # includes virtual orbs too!
#         C_loc_occ =C_loc[:,full_system_scf.mo_occ>0]
        
#     elif localization_method.lower() == 'boys':
#         ### Boys
#         boys_SCF = lo.boys.Boys(full_system_mol, full_system_scf.mo_coeff)
#         C_loc  = boys_SCF.kernel()
#         C_loc_occ =C_loc[:,full_system_scf.mo_occ>0]
        
#     elif localization_method.lower() == 'ibo':
#         ### intrinsic bonding orbs
#         #
#         mo_occ = full_system_scf.mo_coeff[:,full_system_scf.mo_occ>0]
#         iaos = lo.iao.iao(full_system_scf.mol, mo_occ)
#         C_loc_occ = lo.ibo.ibo(full_system_scf.mol, mo_occ, locmethod='IBO', iaos=iaos)#.kernel()
#     else:
#         raise ValueError(f'unknown localization method {localization_method}')
        
    
#     # find indices of AO of active atoms
#     ao_active_inds = np.arange(AO_slice_matrix[0,2], AO_slice_matrix[N_active_atoms-1,3])
#     C_loc_ortho = S_half @ C_loc_occ # <--- orthogonal!!!!
    
    
#     MO_AO_overlap = C_loc_ortho  #  < ϕ_AO_i | ψ_MO_j >
#     MO_active_AO_overlap = np.einsum('ij->j', MO_AO_overlap[ao_active_inds]) # sum over rows of active AOs of MOs!
    
#     print('\noverlap:', MO_active_AO_overlap)
#     print(f'threshold for active part: {THRESHOLD} \n')
    
#     active_MO_inds = np.where(MO_active_AO_overlap>THRESHOLD)[0]
#     enviro_MO_inds = np.array([i for i in range(C_loc_occ.shape[1]) if i not in active_MO_inds]) # get all non active MOs
    
#     active_C = deepcopy(C_loc_ortho)
# #     active_C[enviro_MO_inds, :] = np.zeros((len(enviro_MO_inds), C_loc_ortho.shape[1]))
#     active_C[:, enviro_MO_inds] = np.zeros((C_loc_ortho.shape[0], len(enviro_MO_inds)))
    
#     enviro_C = deepcopy(C_loc_ortho)
# #     enviro_C[active_MO_inds, :] = np.zeros((len(active_MO_inds), C_loc_ortho.shape[1]))
#     enviro_C[:, active_MO_inds] = np.zeros((C_loc_ortho.shape[0], len(active_MO_inds)))
    
#     S_neg_half = sp.linalg.fractional_matrix_power(S_mat, -0.5)
# #     act_orbitals =  S_neg_half @ active_C
# #     env_orbitals =  S_neg_half @ enviro_C
#     act_orbitals =  S_neg_half @ C_loc_ortho[:, active_MO_inds]
#     env_orbitals =  S_neg_half @ C_loc_ortho[:, enviro_MO_inds]
    
    
#     # define active MO orbs and environment
    
# #     Vdag, sing, V = np.linalg.svd(S_mat, full_matrices=True)
# #     S_half = sp.linalg.fractional_matrix_power(S_mat, 0.5)
    
# #     S_neg_half = sp.linalg.fractional_matrix_power(S_mat, -0.5)
    
# #     active_S_neg = deepcopy(S_neg_half)
# #     active_S_neg[enviro_MO_inds, :] = np.zeros((len(enviro_MO_inds), S_neg_half.shape[1]))
# #     active_S_neg[:, enviro_MO_inds] = np.zeros((S_neg_half.shape[1], len(enviro_MO_inds)))
    
# #     enviro_S_neg = deepcopy(S_neg_half)
# #     enviro_S_neg[active_MO_inds, :] = np.zeros((len(active_MO_inds), S_neg_half.shape[1]))
# #     enviro_S_neg[:, active_MO_inds] = np.zeros((S_neg_half.shape[1], len(active_MO_inds)))
    
# #     print(active_S_neg.shape)
# #     print(C_loc_ortho.shape)
# #     print(S_half.shape)
    
# #     act_orbitals =  active_S_neg @ C_loc_ortho  # S_neg_half[active_MO_inds, :] @ C_loc_ortho  # active_S_neg @ C_loc_ortho 
# #     env_orbitals =  enviro_S_neg @ C_loc_ortho # S_neg_half[enviro_MO_inds, :] @ C_loc_ortho  #enviro_S_neg @ C_loc_ortho 

#     C_matrix_all_localized_orbitals = C_loc_occ#@ S_neg_half

#     n_act_mos = len(active_MO_inds)
#     n_env_mos = len(enviro_MO_inds)
    

    
# # ### Get active and enviro density matrices

# dm_active =  2 * act_orbitals @ act_orbitals.T
# dm_enviro =  2 * env_orbitals @ env_orbitals.T

# print(f'number of active MOs: {n_act_mos}')
# print(f'number of enviro MOs: {n_env_mos} \n')


# bool_flag_electron_number = np.isclose(np.trace(dm_active@S_mat) + np.trace(dm_enviro@S_mat), full_system_mol.nelectron)
# print(f'N_active + N_environment = N is: {bool_flag_electron_number}')

# dm_localised_full_system = 2* C_matrix_all_localized_orbitals@ C_matrix_all_localized_orbitals.conj().T
# density_flag = np.allclose(dm_localised_full_system, dm_active + dm_enviro)
# print(f'y_active + y_enviro = y_total is: {density_flag}')

# np.diag(S_neg_half.T @ S_mat @ S_neg_half)

In [21]:
from pyscf.tools import cubegen
import os

def plot_orbital(full_system_mol, C_matrix, index, xyz_geom):

    cube_file = f'temp_MO_orbital_index{index}.cube'    
    cubegen.orbital(full_system_mol, cube_file, C_matrix[:, index])
    

    view = py3Dmol.view(width=width, height=height)
    view.addModel(xyz_geom, "xyz")
    view.setStyle({'stick':{}})

    with open(cube_file, 'r') as f:
        view.addVolumetricData(f.read(), "cube", {'isoval': -0.02, 'color': "red", 'opacity': 0.75})
    with open(cube_file, 'r') as f2:
        view.addVolumetricData(f2.read(), "cube", {'isoval': 0.02, 'color': "blue", 'opacity': 0.75})
    view.zoomTo()
    
    os.remove(cube_file) # delete file once orbital is drawn
    
    return view.show()


In [22]:
# # print active orbitals

# for active_MO_ind in active_MO_inds:
#     plot_orbital(full_system_mol, C_matrix_all_localized_orbitals, active_MO_ind, xyz_geom)

In [23]:
# # print environment orbitals

# for enviro_MO_ind in enviro_MO_inds:
#     plot_orbital(full_system_mol, C_matrix_all_localized_orbitals, enviro_MO_ind, xyz_geom)

# 3. Calculate cross subsystem terms

In [24]:
def Get_energy_and_matrices_from_dm(dm_matrix, scf_obj, check_E_with_pyscf=True):
    """
    Get Energy from denisty matrix

    Note this uses the standard hcore (NO embedding potential here!)
    """

    # It seems that PySCF lumps J and K in the J array 
    J_mat = scf_obj.get_j(dm = dm_matrix)
    K_mat = np.zeros_like(J_mat)
    two_e_term =  scf_obj.get_veff(dm=dm_matrix)
    e_xc = two_e_term.exc
    v_xc = two_e_term - J_mat 

    Energy_elec = np.einsum('ij, ij', dm_matrix, scf_obj.get_hcore() + J_mat/2) + e_xc

    if check_E_with_pyscf:
        Energy_elec_pyscf = scf_obj.energy_elec(dm=dm_matrix)[0]
        if not np.isclose(Energy_elec_pyscf, Energy_elec):
            raise ValueError('Energy calculation incorrect')

    return Energy_elec, J_mat, K_mat, e_xc, v_xc

In [25]:
# use active density
E_act, J_act, K_act, e_xc_act, v_xc_act = Get_energy_and_matrices_from_dm( dm_active, # <- change
                                                                             full_system_scf, 
                                                                             check_E_with_pyscf=True)

In [26]:
# use enviro density
E_env, J_env, K_env, e_xc_env, v_xc_env = Get_energy_and_matrices_from_dm( dm_enviro, # <- change
                                                                             full_system_scf, 
                                                                             check_E_with_pyscf=True)

In [27]:
# cross terms!

j_cross = 0.5 * ( np.einsum('ij,ij', dm_active, J_env) + np.einsum('ij,ij', dm_enviro, J_act) )
k_cross = 0.0

xc_cross = e_xc_total - e_xc_act - e_xc_env
two_e_cross = j_cross + k_cross + xc_cross

In [28]:
print(f'E_active: {E_act}')
print(f'E_enviro: {E_env}')
print(f'E_cross: {two_e_cross}')

E_active: -26.425476634147486
E_enviro: -76.15156912324795
E_cross: 17.996601263382317


# 4. Define V_embed

$$h^{A_{\text{act}} \text{ in } B_{\text{env}}} = h^{\text{core}} + v^{\text{embed}}$$

$$v^{\text{embed}} = g[\gamma^{A_{\text{act}}} + \gamma^{B_{\text{env}}} ] - g[\gamma^{A_{\text{act}}}] + P_{\text{projector}}$$

In [29]:
projector_method = 'huzinaga'
# projector_method = 'mu_shfit'

if projector_method == 'huzinaga':
    Fock = full_system_scf.get_hcore() + full_system_scf.get_veff(dm=dm_active + dm_enviro)
    F_gammaB_S = Fock @ dm_enviro @ S_mat
    projector = -0.5 * (F_gammaB_S + F_gammaB_S.T)
elif projector_method == 'mu_shfit':
    mu = 1e6
    projector = mu * (S_mat @ dm_enviro  @ S_mat)
else:
    raise ValueError(f'Unknown projection method {projector_method}')



In [30]:
# define the embedded term
g_A_and_B = full_system_scf.get_veff(dm=dm_active+dm_enviro)

g_A = full_system_scf.get_veff(dm=dm_active)

v_emb = g_A_and_B - g_A + projector

In [31]:
# PsiEmbed definition
v_emb2 = (J_env + v_xc_total - v_xc_act + projector)

In [32]:
# check these are the SAME!
np.allclose(v_emb, v_emb2)

True

# 5. Run RKS DFT of full system with $V_{emb}$ to get $\gamma_{emb}^{\text{active}}$ 

In [33]:
full_system_mol_EMBEDDED = gto.Mole(atom= geometry,
                      basis=basis,
                       charge=0,
                       #spin=0,
                      )
full_system_mol_EMBEDDED.build()

# RE-DEFINE number of electrons in system
full_system_mol_EMBEDDED.nelectron = 2*n_act_mos

EMBEDDED_full_system_scf = scf.RKS(full_system_mol_EMBEDDED)
EMBEDDED_full_system_scf.verbose=1
EMBEDDED_full_system_scf.max_memory= 8_000
EMBEDDED_full_system_scf.conv_tol = 1e-6
EMBEDDED_full_system_scf.xc = low_level_xc_functional_or_HF

h_core = EMBEDDED_full_system_scf.get_hcore()

# overwrite h_core to include embedding term!!!!
EMBEDDED_full_system_scf.get_hcore = lambda *args: v_emb + h_core


E_emb = EMBEDDED_full_system_scf.kernel()

print(f'embedded Energy: {E_emb}')

embedded Energy: 0.9132799033380543


In [34]:
EMBEDDED_full_system_scf.conv_check

True

In [35]:
EMBEDDED_occ_orbs = EMBEDDED_full_system_scf.mo_coeff[:,EMBEDDED_full_system_scf.mo_occ>0]

# optimized embedded denisty matrix
density_emb = 2 * EMBEDDED_occ_orbs @ EMBEDDED_occ_orbs.conj().T

## check number of electrons makes sense:
electron_check = np.isclose(np.trace(density_emb@S_mat), 2*n_act_mos)

print(f'number of e- in gamma_embedded is correct: {electron_check}')

number of e- in gamma_embedded is correct: True


In [36]:
# calculate embedding correction term

dm_correction = np.einsum('ij, ij', v_emb, density_emb-dm_active)
WF_correction = np.einsum('ij, ij', v_emb, dm_active)

print(f'RKS correction: {dm_correction}')
print(f'WF correction: {WF_correction}')

RKS correction: -0.00020899711496120296
WF correction: 18.21047280584239


# Calculate Energy

The energy can be found according too:

$$E[\gamma^{A_{\text{act}}}_\text{embedded}; \; \; \gamma^{A_{\text{act}}},\mathcal{E}[\gamma^{A_{\text{act}}}_\text{embedded}]
= \mathcal{E}[\gamma^{A_{\text{act}}}_\text{embedded}] + E[\gamma^{B_{\text{env}}}] + g[\gamma^{A_{\text{act}}}, \gamma^{B_{\text{env}}}] + \mathcal{Trace}\big[(\gamma^{A_{\text{act}}}_\text{embedded}-\gamma^{A_{\text{act}}})(h^{A_{\text{act}} \text{ in } B_{\text{env}}} - h )\big]$$



- $\mathcal{E}[\gamma^{A_{\text{act}}}_\text{embedded}]$
    - energy of isolated A computed with new optimized embedded density matrix
    - Note this allows for applicaton of different fucntionals (hence why odd E used)
- $E[\gamma^{B_{\text{env}}}]$ DFT calc of isolated subsystem B
- $g[\gamma^{A_{\text{act}}}, \gamma^{B_{\text{env}}}]$ non additive energy (boundary condition) between subsystems
    - this is obtained from global calc!
- Trace term provides a first order correction to the energy!

In [37]:
## PsiEmbed Way

# J_emb, K_emb =EMBEDDED_full_system_scf.get_jk(dm=density_emb) 

# # note this uses the STANDARD H_core
# e_act_emb = matrix_dot(density_emb, h_core + 0.5 * J_emb - 0.25 * K_emb)
# e_act_emb

In [38]:
## MY way

e_act_emb = full_system_scf.energy_elec(dm=density_emb,
                                        vhf= full_system_scf.get_veff(dm=density_emb),
                                       h1e = h_core)[0]
e_act_emb

-26.42526763365894

In [39]:
e_mf_emb = e_act_emb + E_env + two_e_cross + full_system_scf.energy_nuc() + dm_correction
e_mf_emb # <-- energy from embedded DFT calc

-75.4521607623566

In [40]:
full_system_scf.e_tot # <-- energy from normal DFT calc

-75.45216076573016

In [41]:
np.isclose(e_mf_emb, full_system_scf.e_tot)

True

# 6. High level DFT calc!

In [42]:
full_system_mol_HIGH_LEVEL_DFT = gto.Mole(atom= geometry,
                      basis=basis,
                       charge=0,
                       #spin=0,
                      )

full_system_mol_HIGH_LEVEL_DFT.nelectron = 2*n_act_mos
full_system_mol_HIGH_LEVEL_DFT.build()

full_system_scf_HIGH_LEVEL = scf.RKS(full_system_mol_HIGH_LEVEL_DFT)
full_system_scf_HIGH_LEVEL.verbose=1
full_system_scf_HIGH_LEVEL.max_memory= 8_000
full_system_scf_HIGH_LEVEL.conv_tol = 1e-6
full_system_scf_HIGH_LEVEL.xc = high_level_xc_functional # <-- BETTER functional!


# full_system_scf_HIGH_LEVEL.kernel() < --- do NOT RUN THIS

In [43]:
# run energy calc using high level functional (note density matrix is embedded A)

e_act_emb_HIGH_LVL = full_system_scf_HIGH_LEVEL.energy_elec(dm=density_emb)
e_act_emb_HIGH_LVL 

(-26.529413942127505, 4.032907195833214)

In [44]:
E_high_lvl_DFT = e_act_emb_HIGH_LVL[0] + E_env + two_e_cross + full_system_scf.energy_nuc() + dm_correction
E_high_lvl_DFT # <-- energy from embedded DFT calc

-75.55630707082517

In [45]:
print('High level DFT in DFT error:', np.abs(E_high_lvl_DFT-my_fci.e_tot))
print('LOW level DFT in DFT error:', np.abs(e_mf_emb-my_fci.e_tot))

High level DFT in DFT error: 0.17521512113111726
LOW level DFT in DFT error: 0.2793614295996889


# 7. High level WF calc (classical run) !

In [46]:
full_system_mol_EMBEDDED_HF = gto.Mole(atom= geometry,
                      basis=basis,
                       charge=0,
                       #spin=0,
                      )
full_system_mol_EMBEDDED_HF.build()

# RE-DEFINE number of electrons in system
full_system_mol_EMBEDDED_HF.nelectron = 2*n_act_mos

EMBEDDED_full_system_scf_HF = scf.RHF(full_system_mol_EMBEDDED) # <---- Hartree Fock Calc!
EMBEDDED_full_system_scf_HF.verbose=1
EMBEDDED_full_system_scf_HF.max_memory= 8_000
EMBEDDED_full_system_scf_HF.conv_tol = 1e-6

h_core = EMBEDDED_full_system_scf_HF.get_hcore()

# overwrite h_core to include embedding term!!!!
EMBEDDED_full_system_scf_HF.get_hcore = lambda *args: v_emb + h_core

# EMBEDDED_full_system_scf_HF.kernel()

# print(EMBEDDED_full_system_scf_HF.energy_elec()[0])
# print(EMBEDDED_full_system_scf_HF.energy_tot())

# # manual (should reproduce top result!)
# h1e = EMBEDDED_full_system_scf_HF.get_hcore()
# s1e = EMBEDDED_full_system_scf_HF.get_ovlp()
# vhf = EMBEDDED_full_system_scf_HF.get_veff()
# dm = EMBEDDED_full_system_scf_HF.make_rdm1()
# fock = EMBEDDED_full_system_scf_HF.get_fock(h1e, s1e, vhf, dm)  # = h1e + vhf, no DIIS

# mo_energy, mo_coeff = EMBEDDED_full_system_scf_HF.eig(fock, s1e)
# mo_occ = EMBEDDED_full_system_scf_HF.get_occ(mo_energy, mo_coeff)

# print(EMBEDDED_full_system_scf_HF.energy_elec()[0])
# print(EMBEDDED_full_system_scf_HF.energy_tot())

In [47]:
EMBEDDED_full_system_scf_HF.conv_check

True

In [48]:
# overwrite orbs with RKS embedded orbs!
EMBEDDED_full_system_scf_HF.mo_coeff = EMBEDDED_full_system_scf.mo_coeff 
EMBEDDED_full_system_scf_HF.mo_occ = EMBEDDED_full_system_scf.mo_occ 
EMBEDDED_full_system_scf_HF.mo_energy = EMBEDDED_full_system_scf.mo_energy

print(EMBEDDED_full_system_scf_HF.energy_elec(dm=density_emb)[0])
print(EMBEDDED_full_system_scf_HF.energy_tot(dm=density_emb))
print()
print(EMBEDDED_full_system_scf_HF.energy_elec()[0])
print(EMBEDDED_full_system_scf_HF.energy_tot())

-8.106313378607268
1.0219703496756694

-8.106313378607268
1.0219703496756694


In [49]:
enviro_MO_inds

array([0, 2, 3])

In [50]:
print(EMBEDDED_full_system_scf_HF.mo_energy[enviro_MO_inds])
print(EMBEDDED_full_system_scf_HF.mo_energy[active_MO_inds])

[-0.76528398  0.13185453  0.30363065]
[-0.58386426  0.40171212]


In [51]:
EMBEDDED_full_system_scf_HF.mo_energy

array([-0.76528398, -0.58386426,  0.13185453,  0.30363065,  0.40171212,
        0.5783167 , 17.91170563])

In [52]:
EMBEDDED_full_system_scf_HF.mo_occ

array([2., 2., 0., 0., 0., 0., 0.])

In [53]:
n_env_mos

3

In [54]:
[i for i in range(EMBEDDED_full_system_scf_HF.mol.nao - n_env_mos,
                                           EMBEDDED_full_system_scf_HF.mol.nao)
                         ]

[4, 5, 6]

In [55]:
embedded_cc_obj = cc.CCSD(EMBEDDED_full_system_scf_HF)

embedded_cc_obj.frozen = [i for i in range(EMBEDDED_full_system_scf_HF.mol.nao - n_env_mos,
                                           EMBEDDED_full_system_scf_HF.mol.nao)
                         ]

# embedded_cc_obj.frozen = enviro_MO_inds # as projected out (or pushed to high energy!)!

# embedded_cc_obj.frozen  = np.where(EMBEDDED_full_system_scf_HF.mo_energy>1e5)[0]


# embedded_cc_obj.nocc = EMBEDDED_full_system_scf_HF.mol.nelectron // 2
# embedded_cc_obj.nmo = EMBEDDED_full_system_scf_HF.mo_energy.size
# # NEED to redefine embedded h_core again!
# embedded_cc_obj._scf.get_hcore = lambda *args: v_emb + h_core

e_cc, t1, t2 = embedded_cc_obj.kernel()

CC_flag_check = np.isclose(EMBEDDED_full_system_scf_HF.energy_tot(dm=density_emb),
                          embedded_cc_obj.e_hf)

print(f'\nCC hartree fock energy matches HF embedded calc: {CC_flag_check}!')
embedded_cc_obj.e_hf

E(CCSD) = 1.011232244163489  E_corr = -0.01073810551218026

CC hartree fock energy matches HF embedded calc: True!


1.0219703496756694

In [56]:
print(embedded_cc_obj.frozen)
print(embedded_cc_obj.get_frozen_mask())

[4, 5, 6]
[ True  True  True  True False False False]


In [57]:
# embedded_cc_obj.make_rdm1(ao_repr=False)

In [58]:
# # check CC electron number makes sense!
# rdm1 = embedded_cc_obj.make_rdm1()
# np.isclose(np.trace(rdm), EMBEDDED_full_system_scf_HF.mol.nelectron)

In [59]:
# fci_embed = fci.FCI(EMBEDDED_full_system_scf_HF)
# # fci_embed.frozen = enviro_MO_inds
# fci_embed.run()
# print('E(UHF-FCI) = %.12f' % fci_embed.e_tot)

In [60]:
WF_correction  = np.einsum('ij, ij', v_emb, dm_active) # note different definition

E_WF = embedded_cc_obj.e_hf +e_cc  + E_env + two_e_cross - WF_correction
E_WF 

-75.35420842154453

In [61]:
print('High level DFT in DFT error:', np.abs(E_high_lvl_DFT-my_fci.e_tot))
print('LOW level DFT in DFT error:', np.abs(e_mf_emb-my_fci.e_tot))


print('WF in DFT error:', np.abs(E_WF-my_fci.e_tot))

High level DFT in DFT error: 0.17521512113111726
LOW level DFT in DFT error: 0.2793614295996889
WF in DFT error: 0.3773137704117602


# QC work

$$H_{fermionic} = h_{nuc} + \sum_{p, q} h_{pq} a_{p}^{\dagger} a_{q} + \frac{1}{2} \sum_{p, q ,r ,s} h_{pqrs} a_{p}^{\dagger} a_{q}^{\dagger} a_{r} a_{s}$$

where:

- $$h_{pq} = \int d{\vec{x}} \phi_{p}^{*}({\vec{x}}) \bigg( - \frac{\nabla^{2}_{{\vec{r}}}}{2} - \sum_{I} \frac{Z_{I}}{|{\vec{r}} - {\vec{R}_{I}} |} \bigg) \phi_{q}({\vec{x}})$$


- $$h_{pqrs} = \int d{\vec{x}}_{1} d{\vec{x}}_{2} \frac{\phi_{p}^{*}({\vec{x}}_{1}) \phi_{q}^{*}({\vec{x}}_{2}) \phi_{s}({\vec{x}}_{1}) \phi_{r}({\vec{x}}_{2})}{|{\vec{r}_{1}} - {\vec{r}}_{2}|}$$ 


- $$h_{nuc} = \frac{1}{2} \sum_{I \neq J} \frac{Z_{I} Z_{J}}{| {\vec{R}}_{I} - {\vec{R}}_{J}|}$$

In [62]:
# current_inds = np.hstack((active_MO_inds, enviro_MO_inds))
virtual_inds = np.where(EMBEDDED_full_system_scf.mo_occ==0)[0]
virtual_inds

array([2, 3, 4, 5, 6])

In [63]:
# find virtual inds based on active region!
MO_VIRTUAL_AO_overlap = (S_mat@EMBEDDED_full_system_scf.mo_coeff)[:,virtual_inds]  #  < ϕ_AO_i | ψ_MO_j >
MO_VIRTUAL_AO_overlap_sum = np.einsum('ij->j', np.take(MO_VIRTUAL_AO_overlap, ao_active_inds, axis=0)) # sum over rows of active AOs of MOs!

print('\noverlap:', MO_VIRTUAL_AO_overlap_sum)
print(f'threshold for active part: {THRESHOLD} \n')
MO_inds = np.where(MO_VIRTUAL_AO_overlap_sum>THRESHOLD)[0]
MO_inds = virtual_inds[MO_inds]
MO_inds



overlap: [0.65880066 0.03550248 0.13655585 0.64517299 1.06423127]
threshold for active part: 0.4 



array([2, 5, 6])

In [64]:
EMBEDDED_full_system_scf.mo_occ

array([2., 2., 0., 0., 0., 0., 0.])

In [65]:
#2D array for one-body Hamiltonian (H_core) in the MO representation

H_core_EMBEDDED =  EMBEDDED_full_system_scf_HF.get_hcore()

# # ## occupied full C matrix
# canonical_orbitals_EMBEDDED  = EMBEDDED_full_system_scf.mo_coeff[:,EMBEDDED_full_system_scf.mo_occ>0]

# # # ## full C matrix (inc. virtual orbs)
# canonical_orbitals_EMBEDDED  = EMBEDDED_full_system_scf.mo_coeff

canonical_orbitals_EMBEDDED  = EMBEDDED_full_system_scf.mo_coeff[:,:-n_env_mos]

# # ## zero out
# canonical_orbitals_EMBEDDED  = EMBEDDED_full_system_scf.mo_coeff
# canonical_orbitals_EMBEDDED[:,enviro_MO_inds] = np.zeros((canonical_orbitals_EMBEDDED.shape[0], len(enviro_MO_inds)))

## active MOs only (occupied)
# canonical_orbitals_EMBEDDED  = EMBEDDED_full_system_scf.mo_coeff[:, active_MO_inds]

# # take everything BAR environment indices
# inds_active_occ_and_ALL_virt = np.array([i for i in range(EMBEDDED_full_system_scf.mo_coeff.shape[1]) if i not in enviro_MO_inds])
# canonical_orbitals_EMBEDDED  = EMBEDDED_full_system_scf.mo_coeff[:, inds_active_occ_and_ALL_virt]

# take thresholded virtual indices (based on overlap with active system!)
# canonical_orbitals_EMBEDDED  = EMBEDDED_full_system_scf.mo_coeff[:, np.hstack((active_MO_inds, MO_inds))]

one_body_integrals = canonical_orbitals_EMBEDDED.conj().T @ H_core_EMBEDDED @ canonical_orbitals_EMBEDDED
one_body_integrals.shape

(4, 4)

In [66]:
# canonical_orbitals_EMBEDDED  = EMBEDDED_full_system_scf.mo_coeff
# canonical_orbitals_EMBEDDED[:, enviro_MO_inds] = np.zeros((canonical_orbitals_EMBEDDED.shape[0], len(enviro_MO_inds)))

In [67]:
# A 4-dimension array for electron repulsion integrals in the MO
# representation.  The integrals are computed as
# h[p,q,r,s]=\int \phi_p(x)* \phi_q(y)* V_{elec-elec} \phi_r(y) \phi_s(x) dxdy
from pyscf import ao2mo

eri = ao2mo.kernel(EMBEDDED_full_system_scf_HF.mol, canonical_orbitals_EMBEDDED)
n_orbitals = canonical_orbitals_EMBEDDED.shape[1]
eri = ao2mo.restore(1, # no permutation symmetry
                      eri, 
                    n_orbitals)


two_body_integrals=eri
two_body_integrals = np.asarray(eri.transpose(0, 2, 3, 1), order='C') # change to physists ordering
two_body_integrals.shape

(4, 4, 4, 4)

In [68]:
eri_ao = EMBEDDED_full_system_scf_HF.mol.intor('int2e')

tmp = np.einsum('pi,pqrs->iqrs', canonical_orbitals_EMBEDDED, eri_ao, optimize=True)
tmp = np.einsum('qa,iqrs->iars', canonical_orbitals_EMBEDDED, tmp, optimize=True)
tmp = np.einsum('iars,rj->iajs', tmp, canonical_orbitals_EMBEDDED, optimize=True)
ERI_mo_basis = np.einsum('iajs,sb->iajb', tmp, canonical_orbitals_EMBEDDED, optimize=True)
ERI_mo_basis = np.asarray(ERI_mo_basis.transpose(0, 2, 3, 1), order='C') # change to physists ordering

np.allclose(two_body_integrals, ERI_mo_basis)

True

In [69]:
from pyscf.mp.mp2 import get_frozen_mask

print(embedded_cc_obj.frozen)
get_frozen_mask(embedded_cc_obj)

[4, 5, 6]


array([ True,  True,  True,  True, False, False, False])

In [70]:
# np.allclose(embedded_cc_obj.ao2mo().oooo, ERI_mo_basis[:3,:3,:3,:3])

In [71]:
# inds_active_occ_and_ALL_virt = np.array([i for i in range(EMBEDDED_full_system_scf.mo_coeff.shape[1]) 
#                                          if i not in enviro_MO_inds])

# two_pdm = np.zeros_like(two_body_integrals)

# two_pdm[inds_active_occ_and_ALL_virt,
#         inds_active_occ_and_ALL_virt,
#         inds_active_occ_and_ALL_virt,
#         inds_active_occ_and_ALL_virt] = two_body_integrals[inds_active_occ_and_ALL_virt,
#                                                             inds_active_occ_and_ALL_virt,
#                                                             inds_active_occ_and_ALL_virt,
#                                                             inds_active_occ_and_ALL_virt]

# # Interaction between frozen and active space.
# for p in enviro_MO_inds:
#     # p loops over frozen spatial orbitals.
#     for r in inds_active_occ_and_ALL_virt:
#         for s in inds_active_occ_and_ALL_virt:
#             two_pdm[p,p,r,s] += 2.0*one_body_integrals[p,p]
#             two_pdm[r,s,p,p] += 2.0*one_body_integrals[p,p]
#             two_pdm[p,r,s,p] -= one_body_integrals[p,p]
#             two_pdm[r,p,p,s] -= one_body_integrals[p,p]
            
            
# for i in enviro_MO_inds:
#     for j in enviro_MO_inds:
#         two_pdm[i,i,j,j] += 4.0
#         two_pdm[i,j,j,i] += -2.0

In [72]:
enviro_MO_inds

array([0, 2, 3])

In [73]:
# two_body_integrals = two_pdm
# one_body_integrals= np.einsum('ikjj->ik', two_body_integrals)
# # one_body_integrals = one_body_integrals / full_system_mol.nelectron #(numpy.sum(nelec)-1)

In [74]:
two_body_integrals.shape

(4, 4, 4, 4)

In [75]:
core_constant = 0.0


# # #### code freezes on active space!

# # ### take everything BAR environment indices
# # inds_active_occ_and_ALL_virt = np.array([i for i in range(EMBEDDED_full_system_scf.mo_coeff.shape[1]) if i not in enviro_MO_inds])
# # active_indices =inds_active_occ_and_ALL_virt
# # occupied_indices = enviro_MO_inds
# # occupied_indices = None

# active_indices = np.array([i for i in range(EMBEDDED_full_system_scf.mo_coeff.shape[1])])
# occupied_indices=enviro_MO_inds
                           
# # # # ## take thresholded virtual indices (based on overlap with active system!)
# # # # active_indices = np.hstack((active_MO_inds, MO_inds))

# # # # active_indices = active_MO_inds

# # # occupied_indices = None

# occupied_indices = [] if occupied_indices is None else occupied_indices
# if (len(active_indices) < 1):
#     raise ValueError('Some active indices required for reduction.')

# # Determine core constant
# core_constant = 0.0
# for i in occupied_indices:
#     core_constant += 2 * one_body_integrals[i, i]
#     for j in occupied_indices:
#         core_constant += (2 * two_body_integrals[i, j, j, i] -
#                           two_body_integrals[i, j, i, j])

# # Modified one electron integrals
# one_body_integrals_new = np.copy(one_body_integrals)
# for u in active_indices:
#     for v in active_indices:
#         for i in occupied_indices:
#             one_body_integrals_new[u, v] += (
#                 2 * two_body_integrals[i, u, v, i] -
#                 two_body_integrals[i, u, i, v])

# # Restrict integral ranges and change M appropriately

# one_body_integrals_new = one_body_integrals_new[np.ix_(active_indices, active_indices)]
# two_body_integrals_new = two_body_integrals[np.ix_(active_indices, active_indices, active_indices, active_indices)]
# print(one_body_integrals_new.shape)
# print(two_body_integrals_new.shape)
# print(core_constant)

# one_body_integrals = one_body_integrals_new
# two_body_integrals = two_body_integrals_new


In [76]:
two_body_integrals.shape

(4, 4, 4, 4)

In [77]:
# frozen_inds = [i for i in range(EMBEDDED_full_system_scf_HF.mol.nao - n_env_mos,
#                                            EMBEDDED_full_system_scf_HF.mol.nao)
#                          ]

# frozen_inds

# occupied_frozen = np.intersect1d(np.where(EMBEDDED_full_system_scf_HF.mo_occ>0)[0], frozen_inds)
# print(frozen_inds)
# print(occupied_frozen)

In [78]:
# from openfermion.ops.representations.interaction_operator import get_active_space_integrals

# core_constant, one_body_integrals, two_body_integrals = get_active_space_integrals(
#                                                                     one_body_integrals,
#                                                                    two_body_integrals,
#                                                                    occupied_indices=None,
#                                                                    active_indices=frozen_inds)
# #np.array([i for i in range(EMBEDDED_full_system_scf.mo_coeff.shape[1]) if i not in enviro_MO_inds]))


In [79]:
two_body_integrals.shape

(4, 4, 4, 4)

In [80]:
from openfermion.chem.molecular_data import spinorb_from_spatial

one_body_terms, two_body_terms = spinorb_from_spatial(one_body_integrals, two_body_integrals)

In [81]:
# n_qubits = 2*one_body_integrals.shape[0]

# one_body_terms = np.zeros((n_qubits, n_qubits))
# two_body_terms = np.zeros((n_qubits, n_qubits, n_qubits, n_qubits))

# for p in range(n_qubits//2):
#     for q in range(n_qubits//2):

#         one_body_terms[2*p, 2*q] = one_body_integrals[p,q] # spin UP
#         one_body_terms[(2*p + 1), (2*q +1)] = one_body_integrals[p,q] # spin DOWN

#         # continue 2-body terms
#         for r in range(n_qubits//2):
#             for s in range(n_qubits//2):

#                 ### SAME spin                
#                 two_body_terms[2*p, 2*q , 2*r, 2*s] = two_body_integrals[p,q,r,s] # up up up up
#                 two_body_terms[(2*p+1), (2*q +1) , (2*r + 1), (2*s +1)] = two_body_integrals[p,q,r,s] # down down down down

#                 ### mixed spin                
#                 two_body_terms[2*p, 2*q , (2*r + 1), (2*s +1)] = two_body_integrals[p,q,r,s] # up up down down
#                 two_body_terms[(2*p+1), (2*q +1) , 2*r, 2*s] = two_body_integrals[p,q,r,s] # down down up up            

# ### remove vanishing terms
# EQ_Tolerance=1e-8
# one_body_terms[np.abs(one_body_terms)<EQ_Tolerance]=0
# two_body_terms[np.abs(two_body_terms)<EQ_Tolerance]=0

In [82]:
# np.allclose(one_body_coefficients, one_body_terms)
# np.allclose(two_body_terms, two_body_coefficients)

In [83]:
# two_body_terms = two_body_coefficients

In [84]:
from openfermion.transforms import jordan_wigner
from openfermion.ops import FermionOperator

In [85]:
H_fermionic = FermionOperator((),  full_system_scf.energy_nuc() + core_constant)
# two_body_terms = two_body_terms.transpose(0,2,3,1) # for physist notation!

# one body terms
for p in range(one_body_terms.shape[0]):
    for q in range(one_body_terms.shape[0]):

        H_fermionic += one_body_terms[p,q] * FermionOperator(((p, 1), (q, 0)))

        # two body terms
        for r in range(two_body_terms.shape[0]):
            for s in range(two_body_terms.shape[0]):

                ####### physist notation
                # (requires:
                #           two_body_terms transpose (0,2,3,1) before loop starts!
                #)
                H_fermionic += 0.5*two_body_terms[p,q,r,s] * FermionOperator(((p, 1), (q, 1), (r,0), (s, 0)))

                ######## chemist notation
#                 H_fermionic += 0.5*two_body_terms[p,q,r,s] * FermionOperator(((p, 1), (r, 1), (s,0), (q, 0)))

In [86]:
len(list(H_fermionic))

1057

In [87]:
from openfermion.linalg import get_sparse_operator
H_sparse = get_sparse_operator(H_fermionic)

In [88]:
eigvals_EMBED, eigvecs_EMBED = sp.sparse.linalg.eigsh(H_sparse, which='SA', k=1)
eigvals_EMBED

array([1.01123009])

In [89]:
WF_correction  = np.einsum('ij, ij', v_emb, dm_active) # note different definition

E_VQE = eigvals_EMBED[0]  + E_env + two_e_cross - WF_correction
E_VQE 

-75.35421057647541

In [90]:
N_electrons_expected = 2*n_act_mos
N_electrons_Q_state = np.binary_repr(np.where(np.abs(eigvecs_EMBED)>1e-2)[0][0]).count('1') 

print(f'expect {N_electrons_expected} electrons')
print(f'quantum state has {N_electrons_Q_state} electrons \n')

print(f'number of electrons correct: {N_electrons_expected == N_electrons_Q_state}')

expect 4 electrons
quantum state has 4 electrons 

number of electrons correct: True


In [91]:
print('High level DFT in DFT error:', np.abs(E_high_lvl_DFT-my_fci.e_tot))
print('LOW level DFT in DFT error:', np.abs(e_mf_emb-my_fci.e_tot))


print('WF in DFT error:', np.abs(E_WF-my_fci.e_tot))
print('VQE error:', np.abs(E_VQE-my_fci.e_tot))

High level DFT in DFT error: 0.17521512113111726
LOW level DFT in DFT error: 0.2793614295996889
WF in DFT error: 0.3773137704117602
VQE error: 0.37731161548087755


In [92]:
from functools import reduce

zero_state = np.array([[1],[0]])
one_state = np.array([[0],[1]])

N = 2*n_act_mos
HF_state = reduce(np.kron, [*[one_state for _ in range(N)], *[zero_state for _ in range(N, two_body_terms.shape[0])]]
                 ) # | 1 1 0 0 >
HF_state.shape


(256, 1)

In [93]:
E = HF_state.conj().T @ H_sparse @ HF_state

print(E[0][0].real)
print(EMBEDDED_full_system_scf_HF.energy_tot(dm=density_emb))
np.isclose(E[0][0].real, EMBEDDED_full_system_scf_HF.energy_tot(dm=density_emb))

1.021970349675673
1.0219703496756694


True

In [94]:
where=np.where(np.around(eigvecs_EMBED, 4)>0)[0]
print(where)

np.binary_repr(where[0]).count('1') 

[ 30  39  51  54  60  75  78  90  99 105 114 120 150 156 165 195 198 204
 216 225]


4

In [95]:
where

array([ 30,  39,  51,  54,  60,  75,  78,  90,  99, 105, 114, 120, 150,
       156, 165, 195, 198, 204, 216, 225])